## 教师信息
    姓名: 董兆宇
    电话/微信: 13426253019 
    邮箱: dongzhaoyu@k2data.com.cn

# Pandas 课程 (2) —— 课程概要：
    1. DataFrame 的 map ,apply, applymap 
    2. Pandas 的cut 和 qcut 用法
    3. Pandas 的 group介绍
    4. DataFrame 关于index的方法介绍
    5. DataFrame 的 数据合并
    6. DataFrame 索引的重命名和排序

## 1. DataFrame 的 map ,apply, applymap 

### 1.1 Pandas 的 map

In [1]:
## 数据源 
import pandas as pd
import numpy as np
df = pd.DataFrame({'house' : list('ABCABAD'),
                   'price' : [100, 90, 100, 100, 120, 150, 200],
                   'toward' : ['1','4','3','','','1','2']})
df

,house,price,toward
0,A,100,1
1,B,90,4
2,C,100,3
3,A,100,
4,B,120,
5,A,150,1
6,D,200,2


In [3]:
## Q 将 toward 为空的地方向上补全并处理成数值型 并且 toward_dict 赋值为东西南北
df.toward[df.toward=='']=np.nan
df.toward = df.toward.fillna(method = 'ffill').astype(int)
# def test(s):
#     toward_dict = {1: '东', 2: '南', 3: '西', 4: '北'}
#     return toward_dict[s]

# df.toward.map(test)  ## map 是可以遍历Series 执行逻辑并聚合结果
## map 的第一个参数是 函数, 即遍历Series 的每个元素 需要执行的函数 , 该函数的至少有一个输入参数, 即Series 的元素

/root/anaconda3/lib/python3.7/site-packages/pandas/core/ops.py:1167: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


TypeError: invalid type comparison

In [4]:
df.toward.map(toward_dict) ## map 可以直接接

NameError: name 'toward_dict' is not defined

### 1.2 Pandas 的 apply

In [5]:
## 数据源 
import pandas as pd
import numpy as np
df = pd.DataFrame(np.random.randint(0,10,(4, 3)), columns=list('abc'), index=range(4))
df

,a,b,c
0,9,6,1
1,8,8,0
2,5,8,1
3,6,6,4


In [6]:
## Q 生成新的一列 Series , 其取值是每一行中 abc的最大值-最小值
def creat_d(s):
    return s.max()- s.min()
df.apply(creat_d,axis=1)
## apply 是可以遍历DataFrame 和 Series ，执行逻辑并聚合结果
## apply 的第一个参数是 函数, 即遍历DataFrame 的每个行或没列 ,需要执行的函数 , 该函数的至少有一个输入参数
      ## 即ataFrame的 每一行Series 或 每一列Series 
## apply 通过 axis 参数来控制行遍历 还是 列遍历 ，默认是0 即列遍历 

0    8
1    8
2    7
3    2
dtype: int64

In [10]:
## Q 生成新的DataFrame ，共两列 , 分别是 df 中每一行的 1/4分位数和 3/4 分位数

# def creat_per(s):
#     return [np.percentile(s,25),np.percentile(s,75)]

# def creat_per(s):
#     return pd.Series([np.percentile(s,25),np.percentile(s,75)])

def creat_per(s):
#     return pd.Series([np.percentile(s,25),np.percentile(s,75)],index = ['Q1','Q3'])
    return pd.Series([np.percentile(s,25),np.percentile(s,75)],index = ['Q1','Q3'])
    
res  =df.apply(creat_per,axis=1)
res
## apply可以 根据指定的 函数进行处理并聚合 ， 聚合成新的Series或者 DataFrame 

,Q1,Q3
A,20.00,90.25
B,65.00,90.25
C,15.25,37.00
D,15.75,45.75


In [7]:
## 更快的方式解决问题
d = df.max(axis=1)-df.min(axis=1)
res = pd.concat([df.quantile(0.25,axis=1),df.quantile(0.75,axis=1)],axis=1)
res
## 能支持向量运算的 就避免 使用 apply 等操作 ，向量计算 一定是 pandas 中最快的解决方案

,0.25,0.75
0,3.5,7.5
1,4.0,8.0
2,3.0,6.5
3,5.0,6.0


### 1.3 Pandas 的 applymap

In [8]:
## 数据源 
df = pd.DataFrame(np.random.randint(0,100,(4, 4)), columns=[u'物理',u'化学',u'英语',u'语文'], index=[u'A',u'B',u'C',u'D'])
df

,物理,化学,英语,语文
A,20,97,20,88
B,35,89,75,94
C,46,34,18,7
D,21,99,0,28


In [9]:
# Q 将所有同学的分数 用 优，良，中，差体现 
## 优的分数 区间是 [90,100]
## 良的分数 区间是 [80,90)
## 中的分数 区间是 [60,80)
## 差的分数 区间是 [0,60)

def creat_grade(s):
    if s<=100 and s>=90:
        return u'优'
    if s<90 and s>=80:
        return u'良'
    if s<80 and s>=60:
        return u'中'
    if s<60 and s>=0:
        return u'差'
    
df.applymap(creat_grade)
## applymap 是 针对DF 的每一个元素 执行逻辑并聚合结果

,物理,化学,英语,语文
A,差,优,差,良
B,差,良,中,优
C,差,差,差,差
D,差,优,差,差


### 1.4 apply , map , applymap 用法总结 

1. apply作用于DataFrame的列或行 以及 Series的所有元素 , 聚合时可以生成DataFrame 或 Series
2. map作用于Series的所有元素 ， 聚合时可以生成Series 
3. applymap作用于DataFrame的所有元素 ， 聚合时可以生成DataFrame

#### 课上练习题：
1. 将df 的A列 除去首尾的字符 并转化为整数 赋值给新列C
2. 将df 的B列 按照字典d 进行值替换

In [23]:
import re
a = 'sda1239lkda'
b = re.sub(r'\D', '', a)
print(b)

1239


In [26]:
d = {1:'a',7:'c',4:'b',8:'c'}
df = pd.DataFrame(list(zip(['abd234b','a24bf','a12355b','a980s'],[1,8,4,12,10])),columns = ['A','B'])
df

,A,B
0,abd234b,1
1,a24bf,8
2,a12355b,4
3,a980s,12


In [28]:
import re
df['C']=df.A.apply(lambda x:re.sub(r'\D', '', x))
# df['C']=df.A.apply(lambda x:int(x.strip('abs')))
df

,A,B,C
0,abd234b,1,234
1,a24bf,8,24
2,a12355b,4,12355
3,a980s,12,980


In [ ]:
df.B=df.B.map(d)
df

## 2. Pandas 的cut 和 qcut 用法

### 2.1 pd.cut

In [ ]:
## 示范数据源
df = pd.Series(np.random.randint(0,100,size=100))
df.head()

In [ ]:
# Q 按照 (0,20] , (20,50] , (50,90] , (90,100] 对数据进行分组 
# Q 上述分组，组名分别为  ['A','B','C','D']
# Q 统计 按照 [0,20) , [20,50) , [50,90) , [90,100) 对数据进行分组 
cut1 = pd.cut(df,bins = [0,20,50,90,100])
cut2 = pd.cut(df,bins = [0,20,50,90,100],labels=['A','B','C','D'])
cut3 = pd.cut(df,bins = [0,20,50,90,100],right=False)

### 2.2 pd.qcut

In [ ]:
# Q 按照 df的 四个分位数 将数据 分组
# Q 上述分组，组名分别为  ['Q1','Q2','Q3','Q4']
# Q 按照0, 1/6 ,1/4 ,2/3 ,5/6 ,1 分位数 将数据进行分组
cut1 = pd.qcut(df,4) 
cut2 = pd.qcut(df,4,labels= ['Q1','Q2','Q3','Q4'])
cut3  = pd.qcut(df,[0,1/6.0,1/4.0,2/3.0,5/6.0,1])

In [17]:
df = pd.DataFrame(np.random.randint(0,100,(4, 4)), columns=[u'物理',u'化学',u'英语',u'语文'], index=[u'A',u'B',u'C',u'D'])
df

,物理,化学,英语,语文
A,11,84,94,68
B,68,26,16,60
C,93,22,93,73
D,47,41,64,97


In [18]:
df.apply(lambda x: pd.cut(x,bins = [0,60,80,90,101],right=False,labels = [u'差',u'中',u'良',u'优']),axis=1)

,物理,化学,英语,语文
A,差,良,优,中
B,中,差,差,中
C,优,差,优,中
D,差,差,中,优


## 3. Pandas 的 group介绍

### 3.1 group的基本用法

In [37]:
## 示范数据源
data = [['girl', 'uniql', 'sweater', 3],
 ['girl', 'etam', 'suit', 1],
 ['girl', 'etam', 'pants', 1],
 ['girl', 'lagogo', 'jacket', 2],
 ['boy', 'hailan', 'pants', 2],
 ['boy', 'hailan', 't-shirt', 1],
 ['mother', 'hengyuanxiang', 'coat', 2],
 ['mother', 'hengyuanxiang', 'sweater', 1],
 ['mother', 'uniql', 'coat', 1],
 ['father', 'uniql', 't-shirt', 2],
 ['father', 'hailan', 'sweater', 1],
 ['father', 'hailan', 'pants', 3]]
df = pd.DataFrame(data,columns = [u'Name', u'Brand', u'Cloth', u'Count'])
df

,Name,Brand,Cloth,Count
0,girl,uniql,sweater,3
1,girl,etam,suit,1
2,girl,etam,pants,1
3,girl,lagogo,jacket,2
4,boy,hailan,pants,2
5,boy,hailan,t-shirt,1
6,mother,hengyuanxiang,coat,2
7,mother,hengyuanxiang,sweater,1
8,mother,uniql,coat,1
9,father,uniql,t-shirt,2


In [36]:
g4 = df['Name'].groupby(df["Name"]).count()
g4

Name
boy       2
father    3
girl      4
mother    3
Name: Name, dtype: int64

In [ ]:
# Q 统计多少个 不同的Name，并且每个Name 有多少个 
def test(x):
    return len(x)
df.groupby("Name").apply(test)  ## 先将 df按照 Name 列进行分组 , 并统计分组后 每组数据子集的 行数
## group 是针对 DF 或 Series 进行分组 ,常和apply 联合使用,分组后可以以组为单位 执行逻辑并聚合结果

In [ ]:
# Q 统计多少个 不同的Name，并且每个Name 有多少个 

## 方式一 value_counts
g0 = df.Name.value_counts() ## value_counts是 Series下特有的方法, 用于频次统计
## 方式二 groupby , 下面六种写法等价 
g1 = df.groupby("Name")['Count'].count() ## by 可以是 df的字段名,聚合时可以只选取特定列聚合 ,聚合方法可以使用内置函数 count 
g2 = df.groupby("Name").count()['Count'] ## 和上面的区别是 g2是 先计算了所有列的聚合结果，然后取了Count 列
                                         ## 而 g1是 只取了 Count 列，进行聚合
g3 = df.groupby(df["Name"])['Name'].count() ## by 也可以是 Series
g4 = df['Name'].groupby(df["Name"]).count() ## 可以单取某个Series 进行groupby , 但是此时的by 只能by Series (需要同index)
g5 = df[['Name']].groupby("Name").count() ## 其实 df[['Name']] 还是 DataFrame 只是只有Name一列，所以by  可以是 字段名
g6 = df.groupby("Name").apply(len)  ## 聚合时可以使用apply 外接 特定函数

# error  df['Name'].groupby("Name").count()   ## Series 进行groupby 时不能by 字段名
# error  df.groupby(df[["Name"]]).count()   ## Series 进行groupby 时不能by 字段名

In [ ]:
# Q 统计 不同Name中 Count 列的总和
## 下面四种写法等价
## 内置函数
g1 = df.groupby("Name")['Count'].sum() ## 先选取Count列 , 然后对选定列 进行sum求和
g2= df.groupby("Name").sum()['Count'] ## 先全部计算sum求和, 然后选取Count列
## 自定义函数
g3 = df.groupby("Name").apply(lambda x:x['Count'].sum())
def get_Count(g):
    return g['Count'].sum()
g4 = df.groupby("Name").apply(get_Count)

In [38]:
# Q 按照 Name和 Brand 分组 ，统计每个组内有多少种 Cloth
## 下面三种方式均可 
g1 =  df.groupby(['Name','Brand']).apply(lambda x:len(x.Cloth.unique()))
g2 =  df.groupby(['Name','Brand']).apply(lambda x:len(x.Cloth.drop_duplicates())) 
g3 =  df.groupby([df['Name'],df['Brand']]).apply(lambda x:len(x.Cloth.unique()))
## groupby 可以使用多个 columns , groupby 接收的是 list 
## list 的元素可以是 df 的column 的值, 也可以是 Series , 但是还是需要保持同index
# print (type(g3))
g3  ## g3 因为是by 多个字段而来的, 所以聚合后的结果是 多重索引的结果 

Name    Brand        
boy     hailan           2
father  hailan           2
        uniql            1
girl    etam             2
        lagogo           1
        uniql            1
mother  hengyuanxiang    2
        uniql            1
dtype: int64

In [40]:
g3.loc['father']['hailan']
# g3.loc['father'].loc['hailan']  ## 想要在多重索引的DF 中索引元素, 想要一层一层的索引

2

In [42]:
# g3.index.get_level_values(0) 
g3.index.get_level_values(1) ## DF.index 可以通过get_level_values 获取某重索引 全部值
## DF.columns 如果是多重索引, 也具备get_level_values 方法

Index(['hailan', 'hailan', 'uniql', 'etam', 'lagogo', 'uniql', 'hengyuanxiang',
       'uniql'],
      dtype='object', name='Brand')

In [56]:
## 如果数据的index 是多重索引，可以通过reset_index 将index还原成列
g4 = g3.reset_index()  ## 默认将所有重 索引都 还原成列
print (type(g4))
g4 = g4.rename(index = str, columns={0: "a"})
g4

<class 'pandas.core.frame.DataFrame'>


,Name,Brand,a
0,boy,hailan,2
1,father,hailan,2
2,father,uniql,1
3,girl,etam,2
4,girl,lagogo,1
5,girl,uniql,1
6,mother,hengyuanxiang,2
7,mother,uniql,1


In [ ]:
g3.reset_index(0) ## 也可以指定 某重索引 进行还原
g3.reset_index([0,1]) ## 也可以指定 多个 索引 进行还原

### groupby 的用法总结
1. groupby函数 用于对数据集的分组
2. 用于分组的 数据集可以是 DataFrame 也可以是 Series
3. 用于分组的 关键字 可以是 一个字段名，或者是一组字段名，可以是一个Series，也可以是一组Series，但不能是DataFrame
4. 组内的分析 方法 可以直接使用内置方法，例如sum,count,mean,std,max,min ... 
5. 组内的分析 方法 也可以使用apply，接收自定义函数进行分析 并聚合

#### 课上练习题：
1. 以occupation分组，求每一种职业所有用户的平均年龄
2. 求每一种职业男性的占比
3. 获取每一种职业对应的最大和最小的用户年龄, 并生成两列的DataFrame来记录最大和最小年龄的 结果

In [95]:
df = pd.DataFrame([['1', '24', 'F', 'technician', '85711'],['2', '53', 'F', 'other', '94043'],\
 ['3', '23', 'M', 'writer', '32067'],['4', '24', 'M', 'technician', '43537'],\
 ['5', '33', 'F', 'other', '15213'],['6', '42', 'M', 'executive', '98101'],\
 ['7', '57', 'F', 'administrator', '91344'],['8', '36', 'M', 'administrator', '05201'],\
 ['940', '32', 'M', 'administrator', '02215'],['941', '20', 'M', 'student', '97229'],\
 ['942', '48', 'F', 'librarian', '78209'],['943', '22', 'M', 'student', '77841']],\
                  columns = [u'user_id', u'age', u'gender', u'occupation', u'zip_code'])
df[['user_id','age','zip_code']] = df[['user_id','age','zip_code']].astype(int)
df

,user_id,age,gender,occupation,zip_code
0,1,24,F,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213
5,6,42,M,executive,98101
6,7,57,F,administrator,91344
7,8,36,M,administrator,5201
8,940,32,M,administrator,2215
9,941,20,M,student,97229


In [97]:
a = df.gender
print(type(a))

<class 'pandas.core.series.Series'>


In [80]:
a = df.groupby('occupation')['age'].mean()
print(a)

occupation
administrator    41.666667
executive        42.000000
librarian        48.000000
other            43.000000
student          21.000000
technician       24.000000
writer           23.000000
Name: age, dtype: float64


In [111]:
# df1 = df[df.gender == 'M']
a = df[df.gender == 'M'].groupby('occupation')['gender'].count()
b = df.groupby('occupation')['gender'].count()
a/b

occupation
administrator    0.666667
executive        1.000000
librarian             NaN
other                 NaN
student          1.000000
technician       0.500000
writer           1.000000
Name: gender, dtype: float64

In [88]:
df1 = df.groupby('occupation').apply(lambda x: pd.Series([x.age.max(), x.age.min()], index = ['max','min']))
df1

,max,min
occupation,,
administrator,57,32
executive,42,42
librarian,48,48
other,53,33
student,22,20
technician,24,24
writer,23,23


### 4.1 set_index用法

In [112]:
df  = pd.DataFrame([['a',1,2],['a',1,2],['a',2,2],['b',7,3],['b',8,4]],columns = ['A','B','C'])
df

,A,B,C
0,a,1,2
1,a,1,2
2,a,2,2
3,b,7,3
4,b,8,4


In [115]:
df.set_index('B')  ## set_index 可以将DF中每列作为其index , 如果输入的是DF的column , DF将此列变为index

,A,C
B,,
1,a,2
1,a,2
2,a,2
7,b,3
8,b,4


In [ ]:
df1 = df[['B','C']]
df1.index = df1.index+20
df.set_index(df1['B']) ## set_index 也可以将其他DF中的Series 作为index , 必须满足同长度

In [ ]:
df.set_index(['A','B'])  ## set_index 可以将多列作为索引, 执行函数后 得到的是多重索引
df.set_index([df1['B'],df1['C']]) ## set_index 也可以传入一组Series ,但是传入的不能是 DataFrame
df2 = df.set_index('A',drop=False) ##  set_index 中drop 默认是True, 即将column转为index时，删除该列, 如置False, 则不删除
df2 = df2.set_index('B',append=True) ##  set_index 中append 默认是False, 即设置的index会覆盖原始index, 如置True, 则不覆盖而是合并
df2

## 将索引还原成列的 方式为 reset_index , 前文已经 涉及了

### 4.2 reindex用法

In [121]:
df=pd.DataFrame(np.arange(9).reshape(3,3),index=['a','c','d'],columns=['one','two','four'])
df

,one,two,four
a,0,1,2
c,3,4,5
d,6,7,8


In [117]:
## reindex 可以根据输入索引 而重置DF索引
df.reindex(['a','b','c','d','e','f'])  

,one,two,four
a,0.0,1.0,2.0
b,NaN,NaN,NaN
c,3.0,4.0,5.0
d,6.0,7.0,8.0
e,NaN,NaN,NaN
f,NaN,NaN,NaN


In [120]:
## reindex 可以设置 method , 等效于 后面接了 fillna(method='ffill')
df.reindex(['a','b','c','d','e','f'],method='ffill')

,one,two,four
a,0,1,2
b,0,1,2
c,3,4,5
d,6,7,8
e,6,7,8
f,6,7,8


In [123]:
## 可以设置 fill_value, 等效于 后面接了 fillna(100)
## 参数 index 和 columns 可以对 行标签和 列标签同时进行 重置索引
df.reindex(index = ['a','b','c','d','e','f'],columns = ['one','three'], fill_value= 100)

,one,three
a,0,100
b,100,100
c,3,100
d,6,100
e,100,100
f,100,100


## 5.  DataFrame 的 数据合并
pandas处理多组数据的时候往往会要用到数据的合并处理，其中有三种方式，concat、append和merge。

### 5.1 concat

In [ ]:
## 用 concat是一种基本的合并方式。而且concat中有很多参数可以调整
## 其中重要的参数是 axis 来指明合并方向。
## 函数默认axis=0, 0是上下合并 (纵向合并)  /    1是左右合并 (横向合并)

In [145]:
s = pd.Series([1,2],index = ['a','c'])
df = pd.DataFrame([[3,4],[5,6]],index = ['a','b'])
df1 = pd.DataFrame([[7,8],[9,0]],index = ['b','c'])
print (s)
print (df)
print (df1)

a    1
c    2
dtype: int64
   0  1
a  3  4
b  5  6
   0  1
b  7  8
c  9  0


In [146]:
pd.concat([s,df,df1]) ## pd.concat 接收的参数是 list, list 中的每个元素可以是 Series, 也可以是 DF

,0,1
a,1,NaN
c,2,NaN
a,3,4.0
b,5,6.0
b,7,8.0
c,9,0.0


In [147]:
pd.concat([s,df,df1],axis=1) ## axis 控制合并方向, axis=1 为横向合并, 合并的原则是通过索引 进行拼接 , 默认取合并的并集

/root/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,0,0,1,0,1
a,1.0,3.0,4.0,NaN,NaN
b,NaN,5.0,6.0,7.0,8.0
c,2.0,NaN,NaN,9.0,0.0


In [149]:
pd.concat([s,df,df1],axis=1,join='inner') 
# pd.concat([s,df],axis=1,join='inner') ## 通过join 可以控制 合并方式, inner 是取交集
## 还有其他函数, 可以通过帮助文档详细了解其参数

,0,0,1,0,1


### 5.2 append

In [153]:
## append只有纵向合并，没有横向合并
df = pd.DataFrame([[3,4],[5,6]],index = ['a','b'])
df1 = pd.DataFrame([[7,8],[9,0]],index = ['b','c'])
df

,0,1
a,3,4
b,5,6


In [154]:
df1

,0,1
b,7,8
c,9,0


In [155]:
s.append(df)
s

a    1
c    2
dtype: int64

In [ ]:
s.append(df)  ## Series 可以使用
df.append(df1) ## DF 可以使用

### 5.3 merge

In [160]:
## merge 只有横向合并，没有纵向合并
## 根据两组数据中的关键字key来合并（key在两组数据中是完全一致的）
df = pd.DataFrame([[6,4,8],[5,6,7]],index = ['a','b'],columns = ['A','B','C'])
df1 = pd.DataFrame([[4,6],[9,0]],index = ['b','c'],columns = ['B','C'])
print (df)
print (df1)

   A  B  C
a  6  4  8
b  5  6  7
   B  C
b  4  6
c  9  0


In [161]:
pd.merge(df[['A','B']],df1,on = 'B') ## pd.merge 只能对两个DF 进行 merge, 使用关键字进行匹配, 如果关键字两个DF都存在, 使用 on参数

,A,B,C
0,6,4,6


In [ ]:
pd.merge(df,df1,on = ['B','C']) ## on 的字段可以是多个, 即多个column 组成的一组list

In [163]:
pd.merge(df[['A','B']],df1,on = 'B',how = 'left') ## pd.merge 默认取合并的交集, 可以通过how 参数控制, outer, left , right

,A,B,C
0,6,4,6.0
1,5,6,NaN


In [164]:
df

,A,B,C
a,6,4,8
b,5,6,7


In [165]:
df1

,B,C
b,4,6
c,9,0


In [169]:
pd.merge(df,df1,on = 'B')  ## 如果合并时除去关键字外 两个DF有同名的column, 合并后此字段会带有_x,_y标识

,A,B,C_x,C_y
0,6,4,8,6


In [ ]:
pd.merge(df[['A','C']],df1[['B','C']],\
         left_on = 'A',right_on = 'C')   ## 左右DF 的关键字如果不同, 可以通过left_on 和 right_on 指定

In [ ]:
pd.merge(df,df1,left_index=True,right_index=True)  ## 可以通过left_index 和 right_index 指定左右DF 通过index 进行数据合并

####  课上练习题：
以下是两张表, df1 是教师的个人信息表 , df2 是教师授课表
1. 按照 名字和职称 合并两表, 保留所有信息
2. 按照 两张表的 索引进行合并, 仅保留两表index相同部分信息, 相同属性列 仅保留一个

In [170]:
df1=pd.DataFrame({'名字':list('ABCDE'),'性别':['男','女','男','男','女'],\
                 '职称':['副教授','讲师','助教','教授','助教']},index=range(1001,1006))
df1

,名字,性别,职称
1001,A,男,副教授
1002,B,女,讲师
1003,C,男,助教
1004,D,男,教授
1005,E,女,助教


In [171]:
df2=pd.DataFrame({'名字':list('ABDAX'),'课程':['C++','计算机导论','汇编','数据结构','马克思原理'],\
                  '职称':['副教授','讲师','教授','副教授','讲师']},index=[1001,1002,1004,1001,3001])
df2

,名字,课程,职称
1001,A,C++,副教授
1002,B,计算机导论,讲师
1004,D,汇编,教授
1001,A,数据结构,副教授
3001,X,马克思原理,讲师


In [182]:
a = pd.merge(df1, df2, on = ['名字', '职称'], how = 'outer')
print(a)

  名字   性别   职称     课程
0  A    男  副教授    C++
1  A    男  副教授   数据结构
2  B    女   讲师  计算机导论
3  C    男   助教    NaN
4  D    男   教授     汇编
5  E    女   助教    NaN
6  X  NaN   讲师  马克思原理


In [183]:
pd.merge(df1,df2)

,名字,性别,职称,课程
0,A,男,副教授,C++
1,A,男,副教授,数据结构
2,B,女,讲师,计算机导论
3,D,男,教授,汇编


## 6. DataFrame 索引的重命名和排序

### 6.1 索引的重命名

In [140]:
df = pd.DataFrame([[6,4,6],[5,6,7]],index = ['b','a'],columns = ['B','A','C'])
df

,B,A,C
b,6,4,6
a,5,6,7


In [131]:
df1 = df.rename(index = {"a":'d'},columns = {'A':'A1','B':'B1'}) ## DF.rename 可以对DF的索引进行重命名
## index和columns 指定行和列 重命名的 键值对, 即输入的值为 dict
df1

,B1,A1,C
d,6,4,6
b,5,6,7


In [132]:
df1.rename(index = str.upper,columns = str.lower) ## rename 接收的也可以是 函数
## rename 等效于 对 DF的 index 和 columns 进行了map

,b1,a1,c
D,6,4,6
B,5,6,7


### 6.2 索引排序

In [143]:
df.sort_index(axis = 1) ## DF.sort_index 可以控制DF 的索引排序, axis控制是对行索引排序 还是列索引排序 axis=1 即对 列排序

,A,B,C
b,4,6,6
a,6,5,7


In [144]:
df = df[['C','B','A']] ## 自定义排序的话, 可以使用这种方式完成
df

,C,B,A
b,6,6,4
a,7,5,6


In [ ]:
## 将指定列 提到首位 或者 指定位置 , 可以 通过drop再insert 的方式完成
mid = df['A']
df.drop(labels=['A'], axis=1,inplace = True) ## DF.drop 可以去除DF的行或列 
df.insert(0, 'A', mid) ## DF.insert 可以指定位置插入Series 并命名column
df

In [ ]:
## 将指定列放置在末位, 可以drop 再赋值的方式完成
mids = df[['A','B']]
df.drop(labels=['A','B'], axis=1,inplace = True)
df[['A','B']] = mids
df